In [10]:
import csv
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
import os

In [2]:
def get_adjacent_matrix(distance_file: str,
                        num_nodes: int,
                        id_file: str = None,
                        graph_type="connect") -> np.array:
    """
    construct adjacent matrix by csv file   根据PEMS数据集的csv文件来构建邻接矩阵
    :param distance_file: path of csv file to save the distances between nodes  # csv文件路径
    :param num_nodes: number of nodes in the graph   graph中节点个数
    :param id_file: path of txt file to save the order of the nodes     
    :param graph_type: ["connect", "distance"] if use weight, please set distance
    :return:
    """
    A = np.zeros([int(num_nodes), int(num_nodes)])

    if id_file:
        with open(id_file, "r") as f_id:
            node_id_dict = {
                int(node_id): idx
                for idx, node_id in enumerate(f_id.read().strip().split("\n"))
            }

            with open(distance_file, "r") as f_d:
                f_d.readline()
                reader = csv.reader(f_d)
                for item in reader:
                    if len(item) != 3:
                        continue
                    i, j, distance = int(item[0]), int(item[1]), float(item[2])
                    if graph_type == "connect":
                        A[node_id_dict[i], node_id_dict[j]] = 1.
                        A[node_id_dict[j], node_id_dict[i]] = 1.
                    elif graph_type == "distance":
                        A[node_id_dict[i], node_id_dict[j]] = 1. / distance
                        A[node_id_dict[j], node_id_dict[i]] = 1. / distance
                    else:
                        raise ValueError(
                            "graph type is not correct (connect or distance)")
        return A

    with open(distance_file, "r") as f_d:
        f_d.readline()
        reader = csv.reader(f_d)
        for item in reader:
            if len(item) != 3:
                continue
            i, j, distance = int(item[0]), int(item[1]), float(item[2])

            if graph_type == "connect":
                A[i, j], A[j, i] = 1., 1.
            elif graph_type == "distance":
                A[i, j] = 1. / distance
                A[j, i] = 1. / distance
            else:
                raise ValueError(
                    "graph type is not correct (connect or distance)")

    return A


def get_flow_data(flow_file: str) -> np.array:
    """
    parse npz to get flow data  读取npz文件得到交通流数据
    :param flow_file: (N, T, D)
    :return:
    """
    data = np.load(flow_file)
    flow_data = data['data'].transpose([1, 0,
                                        2])[:, :,
                                            0][:, :,
                                               np.newaxis]  # [N, T, D]  D = 1
    return flow_data


class PEMSDataset(Dataset):

    def __init__(self, data_path, num_nodes, divide_days, time_interval,
                 history_length, train_mode):
        """
        load processed data
        :param data_path: ["graph file name" , "flow data file name"], path to save the data file names
        :param num_nodes: number of nodes in graph
        :param divide_days: [ days of train data, days of test data], list to divide the original data
        :param time_interval: time interval between two traffic data records (mins)
        :param history_length: length of history data to be used
        :param train_mode: ["train", "test"]
        """
        self.data_path = data_path
        self.num_nodes = num_nodes
        self.train_mode = train_mode
        self.train_days = divide_days[0]
        self.test_days = divide_days[1]
        self.history_length = history_length  # 6
        self.time_interval = time_interval  # 5 min
        self.one_day_length = int(24 * 60 / self.time_interval)
        self.graph = get_adjacent_matrix(distance_file=data_path[0],
                                         num_nodes=num_nodes)
        self.flow_norm, self.flow_data = self.pre_process_data(
            data=get_flow_data(data_path[1]), norm_dim=1)

    def __len__(self):
        if self.train_mode == "train":
            # return self.train_days * self.one_day_length - self.history_length  # 这里为什么要减掉一个history length，导致最后训练数据的长度为45*24*12-6=12954
            return self.train_days * self.one_day_length  # 这里为什么要减掉一个history length，导致最后训练数据的长度为45*24*12-6=12954
        elif self.train_mode == "test":
            return self.test_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(
                self.train_mode))

    def __getitem__(self, index):  # (x, y), index = [0, L1 - 1]
        if self.train_mode == "train":
            index = index
        elif self.train_mode == "test":
            index += self.train_days * self.one_day_length
        else:
            raise ValueError("train mode: [{}] is not defined".format(
                self.train_mode))

        data_x, data_y = PEMSDataset.slice_data(self.flow_data,
                                                self.history_length, index,
                                                self.train_mode)
        data_x = PEMSDataset.to_tensor(data_x)  # [N, H, D]
        data_y = PEMSDataset.to_tensor(data_y).unsqueeze(1)  # [N, 1, D]
        return {
            "graph": PEMSDataset.to_tensor(self.graph),
            "flow_x": data_x,
            "flow_y": data_y
        }

    @staticmethod
    def slice_data(data, history_length, index, train_mode):
        """
        :param data: np.array, normalized traffic data.
        :param history_length: int, length of history data to be used.
        :param index: int, index on temporal axis.
        :param train_mode: str, ["train", "test"].
        :return:
            data_x: np.array, [N, H, D].
            data_y: np.array [N, D].
        """
        if train_mode == "train":
            start_index = index
            end_index = index + history_length
        elif train_mode == "test":
            start_index = index - history_length
            end_index = index
        else:
            raise ValueError(
                "train model {} is not defined".format(train_mode))

        data_x = data[:, start_index:end_index]
        data_y = data[:, end_index]

        return data_x, data_y

    @staticmethod
    def pre_process_data(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            norm_base: list, [max_data, min_data], data of normalization base.
            norm_data: np.array, normalized traffic data.
        """
        norm_base = PEMSDataset.normalize_base(
            data, norm_dim)  # find the normalize base
        norm_data = PEMSDataset.normalize_data(norm_base[0], norm_base[1],
                                               data)  # normalize data

        return norm_base, norm_data

    @staticmethod
    def normalize_base(data, norm_dim):
        """
        :param data: np.array, original traffic data without normalization.
        :param norm_dim: int, normalization dimension.
        :return:
            max_data: np.array
            min_data: np.array
        """
        max_data = np.max(data, norm_dim,
                          keepdims=True)  # [N, T, D] , norm_dim=1, [N, 1, D]
        min_data = np.min(data, norm_dim, keepdims=True)
        return max_data, min_data

    @staticmethod
    def normalize_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, original traffic data without normalization.
        :return:
            np.array, normalized traffic data.
        """
        mid = min_data
        base = max_data - min_data
        normalized_data = (data - mid) / base

        return normalized_data

    @staticmethod
    def recover_data(max_data, min_data, data):
        """
        :param max_data: np.array, max data.
        :param min_data: np.array, min data.
        :param data: np.array, normalized data.
        :return:
            recovered_data: np.array, recovered data.
        """
        mid = min_data
        base = max_data - min_data

        recovered_data = data * base + mid

        return recovered_data

    @staticmethod
    def to_tensor(data):
        return torch.tensor(data, dtype=torch.float)

In [3]:
def get_flow(filename):
    flow_data = np.load(filename)
    # print(type(flow_data))
    # PEMS数据集的data.npz文件中的data这个文件存放数据
    return flow_data['data']

def read_dataset(dataset):
    data = np.load(dataset)
    print(type(data['data']))
    print(data['data'][0].shape)

In [4]:
dataset_file='/home/zhoujianping/Research/ASTGNN/data/PEMS04/PEMS04.npz'
traffic_flow_data=get_flow_data(dataset_file)   
print(traffic_flow_data.shape)  # (307,16992,1)只取flow这个维度，对tensor做了transform
print(type(traffic_flow_data))


traffic_data = get_flow(dataset_file)
print("data size {}".format(traffic_data.shape))    # (16992,307,3)其中16992代表时间长度 59天*24小时*12次（每5分钟采集一次数据），307是指检测器数量，3是指数据维度分别对应flow，occupy，speed

read_dataset(dataset_file)

(307, 16992, 1)
<class 'numpy.ndarray'>
data size (16992, 307, 3)
<class 'numpy.ndarray'>
(307, 3)


In [5]:
def get_loader(ds_name="PEMS04"):
    num_nodes = 307 if ds_name == 'PEMS04' else 170
    train_data = PEMSDataset(data_path=[
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/distance.csv".format(ds_name),
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/data.npz".format(ds_name)
    ],
                             num_nodes=num_nodes,
                             divide_days=[45, 14],
                             time_interval=5,
                             history_length=6,
                             train_mode="train")

    print(type(train_data))
    print(len(train_data))
    print(type(train_data[0]))
    print(train_data[0]['graph'].shape)
    print(train_data[0]['flow_x'].shape)
    print(train_data[0]['flow_y'].shape)
    
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    test_data = PEMSDataset(data_path=[
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/distance.csv".format(ds_name),
        "/home/zhoujianping/Research/GNN/TrafficPrediction/datasets/{}/data.npz".format(ds_name)
    ],
                            num_nodes=num_nodes,
                            divide_days=[45, 14],
                            time_interval=5,
                            history_length=6,
                            train_mode="test")

    print(len(test_data))
    test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
    return train_loader, test_loader
    
train_loader, test_loader = get_loader('PEMS04')


<class '__main__.PEMSDataset'>
12960
<class 'dict'>
torch.Size([307, 307])
torch.Size([307, 6, 1])
torch.Size([307, 1, 1])
4032


In [6]:
print(len(train_loader))    # math.ceil(45*24*12/64)=203，原始数据集的前45天数据作为训练集，后面14天作为测试集
print(len(test_loader))
for data in train_loader:   # batch_size=64
    # print(data)
    print(type(data))
    print(data.keys())
    print(type(data['graph']))
    print(type(data['flow_x']))
    print(type(data['flow_y']))
    print(data['graph'].shape)
    print(data['flow_x'].shape) # flow_x表示history data，长度取6个
    print(data['flow_y'].shape) # flow_y表示要预测的值
    break

203
63
<class 'dict'>
dict_keys(['graph', 'flow_x', 'flow_y'])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.Size([64, 307, 307])
torch.Size([64, 307, 6, 1])
torch.Size([64, 307, 1, 1])


In [8]:
for data in train_loader:
    print(data['flow_x'].shape)
    print(torch.squeeze(data['flow_x'],dim=3).shape)
    break

torch.Size([64, 307, 6, 1])
torch.Size([64, 307, 6])


In [11]:
class AE(nn.Module):

    def __init__(self):
        # 调用父类方法初始化模块的state
        super(AE, self).__init__()

        # 编码器 ： [b, 784] => [b, 20]
        self.encoder = nn.Sequential(nn.Linear(1842, 256), nn.ReLU(),
                                     nn.Linear(256, 20), nn.ReLU())

        # 解码器 ： [b, 20] => [b, 784]
        self.decoder = nn.Sequential(
            nn.Linear(20, 256),
            nn.ReLU(),
            nn.Linear(256, 1842),
            nn.Sigmoid()  # 图片数值取值为[0,1]，不宜用ReLU
        )

    def forward(self, x):
        """
        向前传播部分, 在model_name(inputs)时自动调用
        :param x: the input of our training model
        :return: the result of our training model
        """
        batch_size = x.shape[0]  # 每一批含有的样本的个数
        # flatten
        # tensor.view()方法可以调整tensor的形状，但必须保证调整前后元素总数一致。view不会修改自身的数据，
        # 返回的新tensor与原tensor共享内存，即更改一个，另一个也随之改变。
        x = x.view(batch_size, 1842)  # 一行代表一个样本

        # encoder
        x = self.encoder(x)

        # decoder
        x = self.decoder(x)

        # reshape
        x = x.view(batch_size, 307, 6)
        return x

In [14]:
# 设备配置
torch.cuda.set_device(0) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [18]:
batch_size = 64
learning_rate = 1e-3    # 学习率

In [19]:
AEmodel = AE().to(device)  # 生成AE模型，并转移到GPU上去
print('The structure of our model is shown below: \n')
print(AEmodel)
loss_function = nn.MSELoss()  # 生成损失函数
optimizer = optim.Adam(AEmodel.parameters(),
                        lr=learning_rate)  # 生成优化器，需要优化的是model的参数，学习率为0.001

The structure of our model is shown below: 

AE(
  (encoder): Sequential(
    (0): Linear(in_features=1842, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=20, bias=True)
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1842, bias=True)
    (3): Sigmoid()
  )
)


In [20]:
# 开始迭代
num_epochs=10
loss_epoch = []
for epoch in range(num_epochs):
    # 每一代都要遍历所有的批次
    for data in train_loader:
        x=torch.squeeze(data['flow_x'],dim=3)
        # [64,307,6]
        x = x.to(device)
        # 前向传播
        x_hat = AEmodel(x)  # 模型的输出，在这里会自动调用model中的forward函数
        loss = loss_function(x_hat, x)  # 计算损失值，即目标函数
        # 后向传播
        optimizer.zero_grad()  # 梯度清零，否则上一步的梯度仍会存在
        loss.backward()  # 后向传播计算梯度，这些梯度会保存在model.parameters里面
        optimizer.step()  # 更新梯度，这一步与上一步主要是根据model.parameters联系起来了

    loss_epoch.append(loss.item())
    if epoch % (num_epochs // 10) == 0:
        print('Epoch [{}/{}] : '.format(epoch, num_epochs), 'loss = ',
                loss.item())  # loss是Tensor类型

Epoch [0/10] :  loss =  0.0058744982816278934
Epoch [1/10] :  loss =  0.004341111984103918
Epoch [2/10] :  loss =  0.004707020707428455
Epoch [3/10] :  loss =  0.004543520975857973
Epoch [4/10] :  loss =  0.0037450729869306087
Epoch [5/10] :  loss =  0.0037430343218147755
Epoch [6/10] :  loss =  0.004415844101458788
Epoch [7/10] :  loss =  0.004109341185539961
Epoch [8/10] :  loss =  0.004412881564348936
Epoch [9/10] :  loss =  0.0037341953720897436
